In [1]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [2]:
loader = CSVLoader('cleaned_judgements.csv')

In [3]:
my_cs=loader.load()

In [4]:
print(my_cs[0])

page_content='Year: 1950
Petitioner: PANNALAL JANKIDAS
Respondent: MOHANLAL AND ANOTHER.' metadata={'source': 'cleaned_judgements.csv', 'row': 0}


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2500,
    chunk_overlap=500,
)

In [8]:
my_chunk=text_splitter.split_documents(my_cs)

In [10]:
my_chunk[0].page_content

'Year: 1950\nPetitioner: PANNALAL JANKIDAS\nRespondent: MOHANLAL AND ANOTHER.'

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [12]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [13]:
vector_store.add_documents(documents=my_chunk)

['00ea11d9-762a-4712-9e60-86a3515dc4cc',
 '9e5a2728-5c0b-432a-8108-7552781bddca',
 'd881c8f8-18cb-4f85-a109-e1960a544c1a',
 '5371c45c-7892-47f9-ac99-dfd0755d1293',
 'c5357b88-4dca-48b1-84d7-29d0eafa2a15',
 '745d9d26-bdb5-4380-9e0a-a16f3408b2cf',
 'c8c8cebf-038c-43ef-8da9-3ce081b6e424',
 '4282592e-35bf-4fae-b63d-5a6602a1b743',
 'ec8379ec-480d-46c2-8c63-51862ab5667c',
 '7bf62b86-a9a3-455a-9b34-d814778f0cf2',
 'b5d8ed48-0dc2-4d81-8865-3c41ac1bcdfb',
 'e5e4155d-57b5-4d6e-a68b-31734a3046fe',
 'b0dbc84d-e998-42e6-ae2a-97f98e06e1d3',
 'b41db18c-2f9d-4ca4-87db-05a9129d5efe',
 'a29690e8-4ef0-48c3-8e02-49910bbe86cc',
 'faaa1f67-17fe-4bfa-9398-d764400945e6',
 '0575415c-9629-4767-beff-e33796dffae1',
 'fd818439-1eae-48a7-b70c-00fda2f8c10a',
 '705a46fd-0642-4d99-a6b9-c3252cf1c47f',
 '3edbd741-b11e-4e25-ba89-a229922cc720',
 '8cde8cd6-83a6-41a4-a48e-130b4ce6f85a',
 '8fb92984-7a63-4260-b6a6-04d28145dc48',
 'bc14197d-3036-47d6-adce-29835974d6e7',
 'e654affa-43cf-4fee-a2ef-22d622a77bb2',
 '663966e2-fea5-

In [63]:
context=vector_store.similarity_search("Year of PANNALAL JANKIDAS and MOHANLAL AND ANOTHER",k=2)

In [64]:
actual_content=context[0].page_content
print(actual_content)

Year: 1950
Petitioner: PANNALAL JANKIDAS
Respondent: MOHANLAL AND ANOTHER.


In [68]:
our_ques="Year of PANNALAL JANKIDAS and MOHANLAL AND ANOTHER"
our_prompt = f"""
You are an expert legal assistant.
Return the answer as a bullet list of case names with their year.
If no relevant information is available, 
Here is the context of some Supreme Court cases:{actual_content}
Our Question:{our_ques}

"""

In [69]:
print(our_prompt)


You are an expert legal assistant.
Return the answer as a bullet list of case names with their year.
If no relevant information is available, 
Here is the context of some Supreme Court cases:Year: 1950
Petitioner: PANNALAL JANKIDAS
Respondent: MOHANLAL AND ANOTHER.
Our Question:Year of PANNALAL JANKIDAS and MOHANLAL AND ANOTHER




In [70]:
from openai import OpenAI

user_input = our_prompt

client = OpenAI(
    base_url="http://127.0.0.1:1234/v1",
    api_key="lm-studio"
)

response = client.chat.completions.create(
    model="mistral-7b-instruct-v0.1",
    messages=[
        {"role": "user", "content": user_input}
    ]
)

print("Assistant:", response.choices[0].message.content)


Assistant: 
* Pannalal Jankidas v. Mohanlal And Another (1950)
